In [1]:
from sklearn.datasets import load_diabetes
import numpy as np
from itertools import product
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.stats import mode

In [2]:
%config InlineBackend.figure_format = 'svg'

## **Understanding the Random forest algorithm**

The Random Forest algorithm uses the concept of bagging (Bootstrap Aggregating) and random feature selection at each split to train multiple decision trees. It then aggregates their outputs (via voting or averaging) to make robust predictions, and it has built-in methods like the OOB error to estimate model performance. The randomness in data and feature selection introduces diversity in the trees, which reduces the risk of overfitting, making Random Forests a powerful ensemble learning method.


### 1. Bootstrapping (Sampling with Replacement)
Each tree in the Random Forest is trained on a random subset of the data, selected with replacement. This technique is called **Bootstrap Aggregating** or **Bagging**.

- Let $S = \{x_1, x_2, ..., x_N\}$ be the original training dataset of size $N$, where $x_i$ represents the $i$-th data point assuming that each data point is a $d$ dimensional vector.
- From $S$, we create $T$ bootstrap samples, denoted by $S_1, S_2, ..., S_T$. Each $S_t$ is generated by randomly selecting $N$ samples from $S$ with replacement. Thus, some samples from $S$ may appear multiple times in $S_t$, while others may not appear at all.
  
### 2. Training Decision Trees
For each bootstrap sample $S_t$, a decision tree $T_t$ is trained independently. The key feature of Random Forest is that, at each split of the decision tree, only a random subset of features is considered rather than all features.

- Let $d$ be the number of features in the dataset.
- At each node of the decision tree, a random subset of $m$ features is chosen to consider for the split, where $m$ is typically $\sqrt{d}$ for classification (or $\log_2 d$ for regression). This ensures that the trees are diverse and reduces the risk of overfitting.

### 3. Prediction from Individual Trees
Once the $T$ decision trees $T_1, T_2, ..., T_T$ are trained, the Random Forest algorithm uses the predictions of these trees to make a final prediction.

#### Classification:
For a new input $x$, each decision tree $T_t$ produces a predicted class label $\hat{y}_t$. The final prediction is made by **majority voting**:

$$
\hat{y} = \text{mode}(\hat{y}_1, \hat{y}_2, ..., \hat{y}_T)
$$

where:
- $\hat{y}_t$ is the predicted class label from tree $t$.
- The mode operation selects the most frequent predicted class across all trees.

#### Regression:
For regression tasks, the prediction from each tree is the continuous output $\hat{y}_t$. The final prediction is the **average** of these predictions:

$$
\hat{y} = \frac{1}{T} \sum_{t=1}^{T} \hat{y}_t
$$

where:
- $\hat{y}_t$ is the predicted value from tree $t$.

### 4. Out-of-Bag (OOB) Error Estimation
Random Forests have a built-in method to estimate the model's performance called the **out-of-bag (OOB) error**. Since each tree is trained on a bootstrap sample, some of the data is left out of the training set for each tree. This data is called the out-of-bag (OOB) data.

- For each data point $x_i$, the OOB error is calculated by using the subset of trees that did not use $x_i$ in their training process to predict the outcome. Let $T_i^{\text{OOB}}$ represent the set of trees that did not use $x_i$ in their training.

- The OOB prediction for $x_i$ is the majority vote (for classification) or the average (for regression) from the trees in $T_i^{\text{OOB}}$. The error is then calculated as the difference between the true value $y_i$ and the OOB prediction.

#### OOB Error (Classification):
$$
\text{OOB Error} = \frac{1}{N} \sum_{i=1}^{N} \mathbb{I}(\hat{y}_i^{\text{OOB}} \neq y_i)
$$
where $\hat{y}_i^{\text{OOB}}$ is the OOB prediction for sample $x_i$, and $\mathbb{I}(\cdot)$ is the indicator function.

#### OOB Error (Regression):
$$
\text{OOB Error} = \frac{1}{N} \sum_{i=1}^{N} (\hat{y}_i^{\text{OOB}} - y_i)^2
$$
where $\hat{y}_i^{\text{OOB}}$ is the OOB predicted value for sample $x_i$.

The OOB error provides a way to estimate the model's generalization error without needing a separate validation set.


## Getting toy data

In [3]:
d = pd.read_csv('heart_failure_clinical_records_dataset.csv')

In [4]:
d.columns

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT'],
      dtype='object')

In [5]:
d.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [6]:
categrical_features=['anaemia','diabetes','high_blood_pressure','sex']

In [7]:
features=['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
        'serum_sodium', 'sex', 'smoking', 'time']

In [8]:
data=np.array(d[features])
target_1=np.array(d['DEATH_EVENT'])
target_2=np.array(d['serum_creatinine'])

In [9]:
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(data, target_1, test_size=0.33, random_state=42)

In [10]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(data, target_2, test_size=0.33, random_state=42)

## Coding the Random forest classifier

In [11]:
## Getting functions for building a decision tree


def split(feature_column, threshold):
    """
    Splits the dataset based on the feature and threshold.
    """
    left_idx = np.where(feature_column <= threshold)[0]
    right_idx = np.where(feature_column > threshold)[0]
    return left_idx, right_idx




def is_pure(y):
    """
    Checks if all targets in the dataset are the same.
    """
    return len(np.unique(y)) == 1



def impurity(y, task):
    """
    Calculates the impurity of a subset.
    """
    if task == "classification":
        # Gini impurity
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)
    elif task == "regression":
        # Variance
        return np.var(y)
    
    
def weighted_impurity(left_y, right_y, y, task):
    """
    Calculates the weighted impurity of a split.
    """
    total_samples = len(left_y) + len(right_y)
    left_weight = len(left_y) / total_samples
    right_weight = len(right_y) / total_samples
    
    return (left_weight * impurity(y[left_y], task)) + (right_weight * impurity(y[right_y], task))


def find_best_split(X, y, task):
    """
    Finds the best feature and threshold to split on.
    
    Parameters:
        X: numpy array of shape (n_samples, n_features)
        y: numpy array of shape (n_samples,)
        task: "classification" or "regression"
    
    Returns:
        feature: index of the best feature
        threshold: value of the best threshold
        left_idx: indices of samples in the left split
        right_idx: indices of samples in the right split
    """
    best_feature, best_threshold = None, None
    best_impurity = float("inf")
    best_left_idx, best_right_idx = None, None
    
    for feature in range(X.shape[1]):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            left_idx, right_idx = split(X[:, feature], threshold)
            
            if len(left_idx) == 0 or len(right_idx) == 0:
                continue
            
            impurity = weighted_impurity(left_idx, right_idx, y, task) 

            ## The higher the impurity is the lower the deltaI as discussed in the formula
            
            if impurity < best_impurity:
                best_feature = feature
                best_threshold = threshold
                best_impurity = impurity
                best_left_idx, best_right_idx = left_idx, right_idx
    
    return best_feature, best_threshold, best_left_idx, best_right_idx






def create_leaf(y, task):
    """
    Creates a leaf node.
    """
    if task == "classification":
        # Majority class
        classes, counts = np.unique(y, return_counts=True)
        return classes[np.argmax(counts)]
    elif task == "regression":
        # Mean value
        return np.mean(y)


    
    

def build_tree(X, y, depth=0, max_depth=7, min_samples_split=2, task="classification"):
    """
    Recursively builds a decision tree.
    
    Parameters:
        X: numpy array of shape (n_samples, n_features)
        y: numpy array of shape (n_samples,)
        depth: current depth of the tree
        max_depth: maximum depth of the tree
        min_samples_split: minimum number of samples required to split
        task: "classification" or "regression"
    
    Returns:
        A tree represented as a nested dictionary or a leaf value.
    """
    
    # Stopping criteria
    if (max_depth is not None and depth >= max_depth) or len(y) < min_samples_split or is_pure(y):
        return create_leaf(y, task)
    
    # Find the best split
    feature, threshold, left_idx, right_idx = find_best_split(X, y, task)
    
    if feature is None:
        return create_leaf(y, task)
    
    # Recursively build left and right branches
    left_tree = build_tree(X[left_idx], y[left_idx], depth + 1, max_depth, min_samples_split, task)
    right_tree = build_tree(X[right_idx], y[right_idx], depth + 1, max_depth, min_samples_split, task)
    
    # Return the decision node
    return {
        "feature": feature,
        "threshold": threshold,
        "left": left_tree,
        "right": right_tree
    }





def predict(sample, tree):
    """
    Predicts the output for a single sample using the tree.
    """
    if not isinstance(tree, dict):  # If tree is not a dictionary object the code runs into this loop
        return tree
    
    feature = tree["feature"]
    threshold = tree["threshold"]
    
    if sample[feature] <= threshold:
        return predict(sample, tree["left"])
    else:
        return predict(sample, tree["right"])

    
    
    
    
    
def predict_all(X, tree):
    """
    Predicts the output for multiple samples.
    """
    return np.array([predict(sample, tree) for sample in X])

# Write your own Random Forest Classifier (Coding Challenge)

A Random Forest is an ensemble learning method that builds multiple decision trees and combines their predictions for better accuracy. Your task is to implement a simplified version of a Random Forest classifier using a predefined decision tree model (see above). Follow the steps below to write the necessary functions:

## 1. Bootstrapping Data  
- Implement a function `get_bootstrapped_data(X, y, size=100)` that selects `size` random samples (with replacement) from the dataset `(X, y)`.  
- Use `np.random.choice()` to generate random indices and return the corresponding samples from `X` and `y`.  
- For simplicity you can retain the entire feature space while generating a bootstrapped sample from the dataset.

## 2. Fitting the Random Forest  
- Implement a function `fit_random_forest(X, y, number_of_trees=300)`.  
- Initialize an empty list to store decision trees.  
- For each tree:
  - Create a bootstrapped dataset using `get_bootstrapped_data()`.  
  - Train a decision tree using `build_tree()` on this dataset.  
  - Store the trained tree in the list.  
- Return the list of trained decision trees.  

## 3. Making Predictions  
- Implement a function `predict_random_forest(X, trees)`.  
- For each decision tree in the forest:  
  - Use `predict_all()` to get predictions on `X`.  
  - Store these predictions in a list or NumPy array.  
- Determine the final prediction for each sample by taking the most common class (mode) across all trees.  
- Return the final predictions.  

 
Step 1: Write the necessary Python functions based on these instructions. 
Step 2: Calculate out-of-bag error for the model.